In [1]:
from imports import *
from datasets.bdd_oia import BDD_OIA

In [2]:
image_home = './data/bdd_oia/lastframe/data/'
label_home = './data/bdd_oia/lastframe/labels/'


In [3]:
# labels = pd.read_csv(label_home+'no_train.csv')
labels = pd.read_pickle(label_home+'no_train.pkl')
labels.head(3)

,file_name,reason,action,reason_lang,reason_lang_ind
7042,746a720a-dfe93574_3.jpg,"[0, 1, 0, 0, 0, 0]","[1, 0, 0, 1]",no lane on the left,"[2, 6, 3, 8, 12, 4, 1]"
14251,b1f62c41-ed0c6521.jpg,"[0, 0, 0, 1, 0, 0]","[0, 1, 1, 0]",obstacles on the rightlane,"[2, 7, 8, 12, 10, 1]"
1988,ab36a641-4db29301.jpg,"[0, 1, 1, 0, 0, 0]","[1, 0, 0, 1]",no lane on the left;solidline on the left,"[2, 6, 3, 8, 12, 4, 0, 11, 8, 12, 4, 1]"


In [4]:
test_img_name = labels.iloc[0]['file_name']
test_img = Image.open(image_home+test_img_name)

In [5]:
import torchvision.transforms as T
mean=[102.9801, 115.9465, 122.7717]
std=[1., 1., 1.]

transform = T.Compose([T.ToTensor(),T.Normalize(mean,std)])

In [39]:
class BDD_OIA_NLP(Dataset):
    def __init__(self, image_root, label_root, ind_to_word_root, image_min_size=120):
        super().__init__()
        import torchvision.transforms as T

        self.image_root = image_root
        self.mean=[102.9801, 115.9465, 122.7717]
        self.std=[1., 1., 1.]

        self._processing(label_root,ind_to_word_root)

        self.transform = T.Compose([T.Resize(image_min_size),
                                    T.ToTensor(),
                                #    T.Normalize(self.mean,self.std),
                                    ])

    def _processing(self, label_root, ind_to_word_root):
        data_df = pd.read_pickle(label_root)

        self.count = len(data_df)
        self.all_images = data_df['file_name']
        self.all_reasons = data_df['reason_lang_ind']
        self.all_actions = data_df['action']
        print("number of samples in dataset:{}".format(self.count))

        with open(ind_to_word_root,'rb') as f:
            self.ind_to_word = pickle.load(f)

        self.num_words = len(self.ind_to_word.keys())

    def __len__(self):
        return self.count

    def __getitem__(self, idx):
        # test = True
        target = {}
        image_name = self.all_images.iloc[idx]

        # print(type(self.all_actions[idx]))
        target['action'] = torch.tensor(self.all_actions.iloc[idx][:4])
        target['reason'] = torch.tensor(self.all_reasons.iloc[idx])

        img_ = Image.open(self.image_root + image_name)

        img = self.transform(img_)

        return img,target

image_dir = './data/bdd_oia/lastframe/data/'
label_dir = './data/bdd_oia/lastframe/labels/'
bdd_oia_dataset = BDD_OIA_NLP(image_dir, label_dir+'no_train.pkl', label_dir+'ind_to_word.pkl',image_min_size=180)

training_loader = DataLoader(bdd_oia_dataset,
                            shuffle=True,
                            batch_size=10,
                            num_workers=0,
                            drop_last=True,
                            collate_fn=utils.collate_fn)
images_batch,labels_batch = next(iter(training_loader))

number of samples in dataset:19936


In [40]:
torch.stack(images_batch).shape

torch.Size([10, 3, 180, 320])

In [41]:
resnet = torchvision.models.resnet50(pretrained=True)

mobilenet = torchvision.models.mobilenet_v2(pretrained=True)

In [42]:
# resnet(torch.stack(images_batch)).shape

In [44]:
from torchsummary import summary

summary(resnet,(3,180,320))

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 64, 90, 160]         9,408
├─BatchNorm2d: 1-2                       [-1, 64, 90, 160]         128
├─ReLU: 1-3                              [-1, 64, 90, 160]         --
├─MaxPool2d: 1-4                         [-1, 64, 45, 80]          --
├─Sequential: 1-5                        [-1, 256, 45, 80]         --
|    └─Bottleneck: 2-1                   [-1, 256, 45, 80]         --
|    |    └─Conv2d: 3-1                  [-1, 64, 45, 80]          4,096
|    |    └─BatchNorm2d: 3-2             [-1, 64, 45, 80]          128
|    |    └─ReLU: 3-3                    [-1, 64, 45, 80]          --
|    |    └─Conv2d: 3-4                  [-1, 64, 45, 80]          36,864
|    |    └─BatchNorm2d: 3-5             [-1, 64, 45, 80]          128
|    |    └─ReLU: 3-6                    [-1, 64, 45, 80]          --
|    |    └─Conv2d: 3-7                  [-1, 256, 45, 80]         16,38

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 64, 90, 160]         9,408
├─BatchNorm2d: 1-2                       [-1, 64, 90, 160]         128
├─ReLU: 1-3                              [-1, 64, 90, 160]         --
├─MaxPool2d: 1-4                         [-1, 64, 45, 80]          --
├─Sequential: 1-5                        [-1, 256, 45, 80]         --
|    └─Bottleneck: 2-1                   [-1, 256, 45, 80]         --
|    |    └─Conv2d: 3-1                  [-1, 64, 45, 80]          4,096
|    |    └─BatchNorm2d: 3-2             [-1, 64, 45, 80]          128
|    |    └─ReLU: 3-3                    [-1, 64, 45, 80]          --
|    |    └─Conv2d: 3-4                  [-1, 64, 45, 80]          36,864
|    |    └─BatchNorm2d: 3-5             [-1, 64, 45, 80]          128
|    |    └─ReLU: 3-6                    [-1, 64, 45, 80]          --
|    |    └─Conv2d: 3-7                  [-1, 256, 45, 80]         16,38

In [46]:
print(summary(mobilenet,(3,180,320)))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 1280, 6, 10]         --
|    └─ConvBNReLU: 2-1                   [-1, 32, 90, 160]         --
|    |    └─Conv2d: 3-1                  [-1, 32, 90, 160]         864
|    |    └─BatchNorm2d: 3-2             [-1, 32, 90, 160]         64
|    |    └─ReLU6: 3-3                   [-1, 32, 90, 160]         --
|    └─InvertedResidual: 2-2             [-1, 16, 90, 160]         --
|    |    └─Sequential: 3-4              [-1, 16, 90, 160]         896
|    └─InvertedResidual: 2-3             [-1, 24, 45, 80]          --
|    |    └─Sequential: 3-5              [-1, 24, 45, 80]          5,136
|    └─InvertedResidual: 2-4             [-1, 24, 45, 80]          --
|    |    └─Sequential: 3-6              [-1, 24, 45, 80]          8,832
|    └─InvertedResidual: 2-5             [-1, 32, 23, 40]          --
|    |    └─Sequential: 3-7              [-1, 32, 23, 40]          10,000
|  

In [47]:
feature_extractor = nn.Sequential(*list(mobilenet.children())[:-1])

In [49]:
summary(feature_extractor,(3,180,320))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 1280, 6, 10]         --
|    └─ConvBNReLU: 2-1                   [-1, 32, 90, 160]         --
|    |    └─Conv2d: 3-1                  [-1, 32, 90, 160]         864
|    |    └─BatchNorm2d: 3-2             [-1, 32, 90, 160]         64
|    |    └─ReLU6: 3-3                   [-1, 32, 90, 160]         --
|    └─InvertedResidual: 2-2             [-1, 16, 90, 160]         --
|    |    └─Sequential: 3-4              [-1, 16, 90, 160]         896
|    └─InvertedResidual: 2-3             [-1, 24, 45, 80]          --
|    |    └─Sequential: 3-5              [-1, 24, 45, 80]          5,136
|    └─InvertedResidual: 2-4             [-1, 24, 45, 80]          --
|    |    └─Sequential: 3-6              [-1, 24, 45, 80]          8,832
|    └─InvertedResidual: 2-5             [-1, 32, 23, 40]          --
|    |    └─Sequential: 3-7              [-1, 32, 23, 40]          10,000
|  

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 1280, 6, 10]         --
|    └─ConvBNReLU: 2-1                   [-1, 32, 90, 160]         --
|    |    └─Conv2d: 3-1                  [-1, 32, 90, 160]         864
|    |    └─BatchNorm2d: 3-2             [-1, 32, 90, 160]         64
|    |    └─ReLU6: 3-3                   [-1, 32, 90, 160]         --
|    └─InvertedResidual: 2-2             [-1, 16, 90, 160]         --
|    |    └─Sequential: 3-4              [-1, 16, 90, 160]         896
|    └─InvertedResidual: 2-3             [-1, 24, 45, 80]          --
|    |    └─Sequential: 3-5              [-1, 24, 45, 80]          5,136
|    └─InvertedResidual: 2-4             [-1, 24, 45, 80]          --
|    |    └─Sequential: 3-6              [-1, 24, 45, 80]          8,832
|    └─InvertedResidual: 2-5             [-1, 32, 23, 40]          --
|    |    └─Sequential: 3-7              [-1, 32, 23, 40]          10,000
|  